Caliberating validation set with test accuracy
Season_1 , Season_2 lag and Seasonal 3 component are the features

In [1]:
%store -r __toolboxforsalesdata

In [2]:
__toolboxforsalesdata

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
import pickle

In [4]:
monitoring_data = pd.read_csv('monitoring_data.csv')

In [5]:
#xgboost model for this calliberation, linear model yet to be obtained.
#Note I am unable to get parameters I set. Must try a diff format.
xgboost_caliberation = pickle.load(open("xgb_model1_set_1_lag_2_seasonal.pickle.dat","rb"))

In [6]:
for_lr = train_calib.dropna()
X = for_lr.drop(['shop_id','item_id','item_category_id','item_cnt_month','unique'], axis = 1)
y = for_lr['item_cnt_month']
lm_calliberation = LinearRegression()
lm_calliberation.fit(X,y)

NameError: name 'train_calib' is not defined

In [ ]:
lm_calliberation_prediction = lm_calliberation.predict(final_val_for_calib.drop('item_cnt_month', axis = 1))

In [ ]:
#Best alpha for ensemble being 0.092,- alpha*lin + (1-alpha)*xgb
final_val_for_calib = valid_calib.dropna()
final_val_for_calib.drop(['shop_id','item_id','item_category_id','unique'], axis = 1, inplace=True)

In [ ]:
xgboost_caliberation_prediction = xgboost_caliberation.predict(xgboost.DMatrix(final_val_for_calib.drop('item_cnt_month', axis = 1)), ntree_limit=211)

In [ ]:
temp = xgboost_caliberation.predict(xg_val, ntree_limit=211)

In [ ]:
pred_call1 = 0.092*lm_calliberation_prediction+(1-0.092)*xgboost_caliberation_prediction
rmse_1 = rmse(final_val_for_calib['item_cnt_month'], pred_call1)
trim_rmse1 = compute_trimmed_rmse(pred_call1.copy(), final_val_for_calib['item_cnt_month'].copy())

In [ ]:
monitoring_data``

In [ ]:
dropped_val_lag1_lag2_season.dropna().shape

In [ ]:
final_val_for_calib.shape

In [ ]:
len(pd.isnull(dropped_val_lag1_lag2_season))/len(val_lag1_2_season)

In [ ]:
rmse(val_lag1_2_season['item_cnt_month'], temp)

In [ ]:
rmse(final_val_for_calib['item_cnt_month'],xgboost_caliberation_prediction)

In [ ]:
trim_rmse1

In [ ]:
monitoring_data

In [ ]:
monitoring_data['Features'] = 'Lag1'

In [ ]:
to_be_added = ['xg_model_6_01_subsample_1', 'lm_model', 'None', 'None', 'alpha(0.092*lm+0.908*xg)', 'ensemble_lin_xgb_0092', 'ensemble_lin_xgb_0092_lag1_lag2_season','Time_based','subset_of_train_full_lag1_lag2_seasonal', '24.935','2.04778','1.09124','Lag1_Lag2_Seasonal']

In [ ]:
monitoring_data = pd.concat([monitoring_data , pd.DataFrame(np.expand_dims(to_be_added,axis = 0) , columns = monitoring_data.columns)], axis = 0)

In [ ]:
pred_call2 = (0.092*lm_calliberation_prediction+(1-0.092)*xgboost_caliberation_prediction)-0.025
pred_call2[pred_call2<0] = 0
rmse_2 = rmse(final_val_for_calib['item_cnt_month'], pred_call2)
trim_rmse2 = compute_trimmed_rmse(pred_call2.copy(), final_val_for_calib['item_cnt_month'].copy())

In [ ]:
rmse(final_val_for_calib['item_cnt_month'], xgboost_caliberation_prediction)

In [ ]:
trim_rmse2

In [ ]:
to_be_added = ['xg_model_6_01_subsample_1', 'lm_model', 'None', 'None', 'alpha(0.092*lm+0.908*xg)', 'ensemble_lin_xgb_0092_less_0025', 'ensemble_lin_xgb_0092_lag1_lag2_season_less_0025','Time_based','subset_of_train_full_lag1_lag2_seasonal', '24.936','2.050922','1.09067','Lag1_Lag2_Seasonal']

In [ ]:
monitoring_data = pd.concat([monitoring_data , pd.DataFrame(np.expand_dims(to_be_added,axis = 0) , columns = monitoring_data.columns)], axis = 0)

In [ ]:
monitoring_data.to_csv('monitoring_data.csv')

In [ ]:
train_full = pd.read_csv('train_full_lag1_lag2_season.csv')

In [ ]:
initial_sales_data1 = pd.read_csv('sales_train.csv')

In [ ]:
initial_sales_data1.head()

In [ ]:
initial_test = pd.read_csv('test.csv')

In [ ]:
initial_sales_data2 = pd.read_csv('item_categories.csv')

In [ ]:
initial_sales_data3 = pd.read_csv('items.csv')

In [ ]:
initial_sales_data2.head()

In [ ]:
monitoring_data.reset_index(inplace = True)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(monitoring_data['Validation_Score_after_clip'], monitoring_data['Public_LB_Score'])
plt.show()

In [ ]:
#May be I should tune the xgboost with current data.
train_full_lag1_2_season = pd.read_csv('train_full_lag1_lag2_season.csv')

In [ ]:
train_full_lag1_2_season.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
monitoring_data

In [ ]:
train_full_lag1_2_season.columns

In [ ]:
train_lag1_2_season, val_lag1_2_season = month_split(train_full_lag1_2_season, 33)

In [ ]:
dropped_train_lag1_lag2_season = train_lag1_2_season.drop(['shop_id','item_id','item_category_id','item_cnt_month','unique'], axis = 1)

In [ ]:
dropped_val_lag1_lag2_season = val_lag1_2_season.drop(['shop_id','item_id','item_category_id','item_cnt_month','unique'], axis = 1)

In [ ]:
len(dropped_train_lag1_lag2_season)

In [ ]:
len(train_full_lag1_2_season['item_cnt_month'])

In [ ]:
#Training with xgboost with default parameters
xg_train = xgboost.DMatrix(dropped_train_lag1_lag2_season,label = train_lag1_2_season.copy()['item_cnt_month'])
xg_val = xgboost.DMatrix(dropped_val_lag1_lag2_season,label = val_lag1_2_season.copy()['item_cnt_month'])

In [ ]:
param_set_2_lag_2_seasonal = {'max_depth': 1, 'eta': 0.05, 'silent': 1, 'objective': 'reg:linear', 'subsample' : 0.5,'eval_metric': 'rmse'}

In [ ]:
progress_1_05_lag1_lag2_seasonal = {}
xg_model_1_05_lag1_lag2_seasonal = xgboost.train(param_set_2_lag_2_seasonal , xg_train,early_stopping_rounds=10,evals = [(xg_train , "train_set"),(xg_val,"validation_set")], evals_result=progress_1_05_lag1_lag2_seasonal ,num_boost_round=1000)

In [ ]:
param_set_3_lag_2_seasonal = {'max_depth': 3, 'eta': 0.05, 'silent': 1, 'objective': 'reg:linear', 'subsample' : 0.5,'eval_metric': 'rmse'}
progress_2_05_lag1_lag2_seasonal = {}
xg_model_2_05_lag1_lag2_seasonal = xgboost.train(param_set_3_lag_2_seasonal , xg_train,early_stopping_rounds=10,evals = [(xg_train , "train_set"),(xg_val,"validation_set")], evals_result=progress_2_05_lag1_lag2_seasonal ,num_boost_round=1000)

In [ ]:
param_set_4_lag_2_seasonal = {'max_depth': 4, 'eta': 0.05, 'silent': 1, 'objective': 'reg:linear', 'subsample' : 0.5,'eval_metric': 'rmse'}
progress_3_05_lag1_lag2_seasonal = {}
xg_model_3_05_lag1_lag2_seasonal = xgboost.train(param_set_4_lag_2_seasonal , xg_train,early_stopping_rounds=30,evals = [(xg_train , "train_set"),(xg_val,"validation_set")], evals_result=progress_3_05_lag1_lag2_seasonal ,num_boost_round=1000)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(np.arange(xg_model_1_05_lag1_lag2_seasonal.best_ntree_limit), progress_1_05_lag1_lag2_seasonal['validation_set']['rmse'][0:xg_model_1_05_lag1_lag2_seasonal.best_ntree_limit], color = 'blue', label = 'xg_depth1_lr_05_subsamp_0.5')
ax1.plot(np.arange(xg_model_2_05_lag1_lag2_seasonal.best_ntree_limit), progress_2_05_lag1_lag2_seasonal['validation_set']['rmse'][0:xg_model_2_05_lag1_lag2_seasonal.best_ntree_limit], color = 'red', label = 'xg_depth2_lr_05_subsamp_0.5')
ax1.plot(np.arange(xg_model_3_05_lag1_lag2_seasonal.best_ntree_limit), progress_3_05_lag1_lag2_seasonal['validation_set']['rmse'][0:xg_model_3_05_lag1_lag2_seasonal.best_ntree_limit], color = 'orange', label = 'xg_depth4_lr_05_subsamp_0.5')
ax1.plot([0,len(progress_1_05_lag1_lag2_seasonal['validation_set']['rmse'])-1], [14,xgboost_caliberation.best_score], color ='green', label = 'xg_depth6_lr_01_subsamp_1')
ax1.legend()
plt.show()

In [ ]:
monitoring_data['row_type_val'] = 'na_dropped'
monitoring_data['row_type_test'] = 'na_predicted_as_zero'

In [ ]:
xg_model_3_05_lag1_lag2_seasonal.best_score

In [ ]:
xg_model_3_05_lag1_lag2_seasonal.best_ntree_limit

In [ ]:
compute_trimmed_rmse(val_lag1_2_season.copy()['item_cnt_month'],xg_model_3_05_lag1_lag2_seasonal.predict(xg_val, ntree_limit=xg_model_3_05_lag1_lag2_seasonal.best_ntree_limit))

In [ ]:
rmse(val_lag1_2_season.copy()['item_cnt_month'],xg_model_3_05_lag1_lag2_seasonal.predict(xg_val))

In [ ]:
why = xg_model_3_05_lag1_lag2_seasonal.predict(xg_val)

In [ ]:
len(np.unique(why))test

In [ ]:
test_for_submit = pd.read_csv('test_full_lag1_lag2_seasonal.csv')

In [ ]:
test_for_submit.drop(['Unnamed: 0','ID','shop_id','item_id','item_category_id','unique'],axis = 1, inplace = True)

In [ ]:
features = dropped_val_lag1_lag2_season.columns.values

In [ ]:
test_for_submit['is_music'] = test_for_submit['is_music'].astype(float)

In [ ]:
xg_test = xgboost.DMatrix(test_for_submit[features])

In [ ]:
try_submitting = xg_model_3_05_lag1_lag2_seasonal.predict(xg_test)

In [ ]:
len(np.unique(try_submitting))/len(try_submitting)

In [ ]:
max(try_submitting)

In [ ]:
min(try_submitting)

In [ ]:
np.mean(try_submitting)

In [ ]:
test_for_submit.columns

In [ ]:
verify = pd.read_csv('test_full_lag1_lag2_seasonal.csv')

In [ ]:
count = 0
for i in test_unique_item_id:
    if i not in train_unique_item_id:
        count+=1
print(count)

In [ ]:
count = 0
for i in test_unique_shop_id:
    if i not in train_unique_shop_id:
        count+=1
print(count)

In [ ]:
train_unique_item_id = np.unique(np.unique(train_full_lag1_2_season['item_id']))

In [ ]:
test_unique_item_id = np.unique(np.unique(verify['item_id']))

In [ ]:
train_unique_shop_id = np.unique(np.unique(train_full_lag1_2_season['shop_id']))

In [ ]:
test_unique_shop_id = np.unique(np.unique(verify['shop_id']))

In [ ]:
len(train_unique_item_id)

In [ ]:
len(test_unique_item_id)

In [ ]:
sum(pd.isnull(test_for_submit[features]['item_price']))

In [ ]:
test_for_submit[test_for_submit["indicator"]==0]

In [ ]:
submit = pd.DataFrame([verify['ID'], try_submitting]).T

In [ ]:
submit['ID'] = submit['ID'].astype(int)
submit.rename(columns={'Unnamed 0': 'item_cnt_month'}, inplace=True)

In [ ]:
submit.loc[submit['item_cnt_month'] >= 20, 'item_cnt_month'] = 20
submit.loc[submit['item_cnt_month'] <= 0, 'item_cnt_month'] = 0

In [ ]:
submit.to_csv('take_care.csv',index=False)

In [ ]:
sum(submit['item_cnt_month'] == 20)

In [ ]:
ax1.d

In [ ]:
fig1 = plt.figure()
ax2 = fig1.add_subplot(111)
sns.distplot(submit['item_cnt_month'].values, ax = ax2)
plt.show()

In [ ]:
dist_comp = pd.read_csv('ensemble_lin_xgb_0092_lag1_lag2_season.csv')

In [ ]:
fig2 = plt.figure(figsize=(5,5))
ax3 = fig2.add_subplot(111)
sns.distplot(dist_comp['item_cnt_month'].values, ax = ax3)
plt.show()

In [ ]:
xg_model_3_05_lag1_lag2_seasonal.best_score

In [ ]:
to_be_added = ['xg_model_4_05_subsample_05', 'None', 'None', 'None', 'None', 'xgb_pred', 'xgb_lag1_lag2_season','Time_based','subset_of_train_full_lag1_lag2_seasonal', '12.5504','2.1850','3.59936','Lag1_Lag2_Seasonal','na_kept_as_is','all_predict_with_xg']

In [ ]:
monitoring_data = pd.concat([monitoring_data , pd.DataFrame(np.expand_dims(to_be_added,axis = 0) , columns = monitoring_data.columns)], axis = 0)

In [ ]:
monitoring_data.drop('index',axis = 1 , inplace=True)

In [ ]:
monitoring_data.reset_index(inplace=True)

In [ ]:
monitoring_data['Test_Prediction_Name'][5] = 'only_xgb_lag1_lag2_season'

In [ ]:
monitoring_data

In [ ]:
submit.to_csv('only_xgb_lag1_lag2_season.csv',index=False)

In [ ]:
??plt.plot

In [ ]:
monitoring_data